In [1]:
from PIL import Image
from create_dataset import create_coco

import matplotlib.pyplot as plt
import os
import numpy as np
import tqdm
import datetime
import json
%matplotlib inline

category2id = {'car':          33,
               'motorbicycle': 34, 
               'bicycle':      35,
               'person':       36, 
               'truck':        38,
               'bus':          39,
               'tricycle':     40,
              }
id2category = {}
for cat, _id in category2id.items():
    id2category[_id] = cat
    
label_name = os.listdir('./Dataset/sample_train_label/')
label_path = ['./Dataset/sample_train_label/'+name for name in label_name]
label_path.sort()
############# To Delete ###############
label_path = label_path[0:2]
############# To Delete ###############

img_name = os.listdir('./Dataset/sample_train_color/')
img_path = ['./Dataset/sample_train_color/'+name for name in img_name]
img_path.sort()
############# To Delete ###############
img_path = img_path[0:2]
############# To Delete ###############

In [2]:
create_coco(img_path, label_path, './Dataset/data.json', crop_size=[75, 75, 28, 28], down_scale=4)

0it [00:00, ?it/s]

Image ID  0 could not find the bounding boxs
Image ID  0 could not find the bounding boxs
Image ID  0 could not find the bounding boxs


2it [00:02,  1.35s/it]
